# Picard iteration on iota to reach zero current
This example runs `GVEC` from a python script multiple times with different parameters and restarting from the final state of each previous run.

The new iota profile that excludes the current contribution is computed from the previous state, fitted to a polynomial and prescribed via the parameterfile to the next run. 

Along the Picard iteration, radial resolution and maximum iteration count is increased, as well.

**Note:** This script needs to `import gvec`, please run it in a virtual environment of python, where the python module `gvec` is already installed!


In [ ]:
import os
from pathlib import Path
import contextlib
import shutil

import numpy as np
import matplotlib.pyplot as plt

# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
import gvec  # using run & modifying the parameters & postprocessing

In [ ]:
@contextlib.contextmanager
def chdir(path: Path | str):
    """
    Contextmanager to change the current working directory.

    Using a context has the benefit of automatically changing back to the original directory when the context is exited, even if an exception is raised.
    """
    path = Path(path)
    old_dir = Path(os.getcwd())

    os.chdir(path)
    yield
    os.chdir(old_dir)

In [ ]:
# total number of gvec runs
iterations = 4  # 12

template = "parameter.ini"
# degree of the polynomial to fit iota-iota_curr
iota_poly_degree = 9
# number of elements during the Picard iterations (gvec_nelems + (gvec_nelems_max-gvec_nelems)*(i/iterations)**2)
gvec_nelems = 2
gvec_nelems_max = 8  # 40
# maximum number of iterations in each GVEC run, min(gvec_maxiter,gvec_miniter* 2**i)
gvec_miniter = 10
gvec_maxiter = 80  # 1000

initLA_at_restart = "F"

profiles = {}

print(f"Toplevel working directory: {os.getcwd()}")
for i in range(0, iterations + 1):
    print(f"Iteration {i}")
    params = {}
    profiles[i] = {}
    restartfile = ""
    # find previous state
    if i > 0:
        previous_path = Path(f"run_{i-1:02d}")
        statefile = sorted(previous_path.glob("*State*.dat"))[-1]
        print(f"Using statefile {statefile}")
        restartfile = Path("..") / statefile

        with gvec.State(previous_path / "parameter.ini", statefile) as state:
            rho = np.sqrt(np.linspace(0, 1, 41)[1:])
            ev = gvec.Evaluations(rho=rho, theta="int", zeta="int", state=state)
            state.compute(ev, "iota", "iota_curr")
            ev_volint = gvec.Evaluations(
                rho="int", theta="int", zeta="int", state=state
            )
            state.compute(ev_volint, "W_MHD")
            print(f"W_MHD={ev_volint.W_MHD:.7e}")
            profiles[i - 1]["ev"] = ev[["rho", "iota", "iota_curr", "I_tor"]]
            profiles[i - 1]["W_MHD"] = ev_volint["W_MHD"]
            iota_coefs = np.polyfit(
                rho**2, ev["iota"] - ev["iota_curr"], iota_poly_degree
            )
            iota_curr_rms = np.sqrt(
                (ev.iota_curr**2).mean("rad").item()
            )  # possible early stop condition
            print(f"iota_curr_rms: {iota_curr_rms:.3e}")

            print(f"New iota parameters: {iota_coefs[::-1]}")

            # set new parameters
            params["init_LA"] = initLA_at_restart
            params["sign_iota"] = 1
            # currently adapt_parameter_file expects strings
            params["iota_coefs"] = "(/" + ", ".join(map(str, iota_coefs[::-1])) + "/)"

    params["maxiter"] = int(np.amin([gvec_maxiter, gvec_miniter * 2**i]))
    params["sgrid_nElems"] = int(
        gvec_nelems + (gvec_nelems_max - gvec_nelems) * (i / iterations) ** 2
    )
    print("-" * 40)
    print(f"Running iteration {i}/{iterations}")

    # prepare the run directory
    path = Path(f"run_{i:02d}")
    if path.exists():
        print(f"Removing existing run directory {path}")
        shutil.rmtree(path)

    if not path.exists():
        path.mkdir()
        print(f"created run directory {path}")

    gvec.util.adapt_parameter_file(template, path / "parameter.ini", **params)

    with chdir(f"run_{i:02d}"):
        gvec.run("parameter.ini", restartfile, stdout_path="stdout.txt")
    print("=" * 40)
    if i == iterations:
        previous_path = Path(f"run_{i:02d}")
        statefile = sorted(previous_path.glob("*State*.dat"))[-1]
        print(f"Using statefile {statefile}")
        with gvec.State(previous_path / "parameter.ini", statefile) as state:
            rho = np.sqrt(np.linspace(0, 1, 40)[1:])
            ev = gvec.Evaluations(rho=rho, theta="int", zeta="int", state=state)
            state.compute(ev, "iota", "iota_curr")
            ev_volint = gvec.Evaluations(
                rho="int", theta="int", zeta="int", state=state
            )
            state.compute(ev_volint, "W_MHD")
            print(f"W_MHD={ev_volint.W_MHD:.7e}")
            profiles[i]["ev"] = ev[["rho", "iota", "iota_curr", "I_tor"]]
            profiles[i]["W_MHD"] = ev_volint["W_MHD"]

print("=" * 40)
print("Picard iteration finished")
print("=" * 40)

In [ ]:
maxiter_plot = iterations

# === Plot evolution of profiles === #
fig, axs = plt.subplots(1, 3, figsize=(15, 5), tight_layout=True, sharex=True)


for var, ax in zip(["I_tor", "iota_curr"], [axs[2], axs[1]]):
    for step in range(0, maxiter_plot + 1):
        col = "blue" if (step == maxiter_plot) else "black"
        alp = 1.0 if (step == maxiter_plot) else 0.2 + 0.3 * (step / maxiter_plot)
        lsty = "-x" if (step == maxiter_plot) else "-"
        ev = profiles[step]["ev"]
        ax.semilogy(
            ev.rho**2, np.abs(ev[var].values) + 1.0e-16, lsty, color=col, alpha=alp
        )
    ax.set(
        title=ev[var].attrs["long_name"],
        xlabel="$\\rho^2$",
        # ylabel=f"$|{ev[var].attrs['symbol']}|$",
    )

sign_iota = -1
for ax in [axs[0]]:
    for step in range(0, maxiter_plot + 1, 1):
        col = "blue" if (step == maxiter_plot) else "black"
        alp = 1.0 if (step == maxiter_plot) else 0.2 + 0.3 * (step / maxiter_plot)
        lsty = "-x" if (step == maxiter_plot) else "-"
        ev = profiles[step]["ev"]
        ax.plot(ev.rho**2, sign_iota * ev["iota"], lsty, color=col, alpha=alp)
        coefs = np.polyfit(ev.rho**2, ev["iota"] - ev["iota_curr"], iota_poly_degree)
        y_fit = np.polyval(coefs, ev.rho**2)
        print(
            f"step={step},fiterr={np.amax(np.abs(ev['iota'].values-ev['iota_curr'].values-y_fit)):.3e}, iota_coefs=(/{', '.join([f'{coef:.8e}' for coef in coefs[::-1]])}/)"
        )

        # ax.plot(ev.rho**2,sign_iota*y_fit,"--",color="red",alpha=0.1+0.9*step/iterations)
    ax.set(
        title=ev["iota"].attrs["long_name"],
        xlabel="$\\rho^2$",
    )

axs[2].set(ylabel=r"$|I_\text{tor}|\quad [A]$")
axs[1].set(ylabel=r"$|\iota_\text{curr}|$")
axs[0].set(ylabel=r"$\iota$")


plt.savefig("current_convergence.pdf", format="pdf", bbox_inches="tight")